In [4]:
# Load Module
import torch
import torch.nn as nn
import cv2

In [3]:
filename = '../DATA/angryping.jpg'

In [6]:
img_np = cv2.imread(filename)/255.0
print(img_np.shape, img_np.ndim)

img_np2 = cv2.resize(img_np, (50, 50))
print(img_np2.shape, img_np2.ndim)

gray_np = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)/255.0
gray_np2 = cv2.resize(gray_np, (50, 50))
print(gray_np.shape, gray_np.ndim, '---', gray_np2.shape, gray_np2.ndim)


(535, 473, 3) 3
(50, 50, 3) 3
(535, 473) 2 --- (50, 50) 2


In [7]:
imshow = cv2.imshow('image', img_np2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
# numpy to tensor
img_tensor = torch.tensor(img_np2, dtype=torch.float32)
gray_tensor = torch.tensor(gray_np2, dtype=torch.float32)
print(img_tensor.shape, img_tensor.ndim, '---', gray_tensor.shape, gray_tensor.ndim)

torch.Size([50, 50, 3]) 3 --- torch.Size([50, 50]) 2


In [12]:
# Pytorch 이미지 데이터 형식 : (C, H, W) -> (H, W, C)
# - permute()
color_hwc = img_tensor.permute(2, 0, 1)
gray_hwc = gray_tensor.unsqueeze(0)
print(color_hwc.shape, color_hwc.ndim, '---', gray_hwc.shape, gray_hwc.ndim)

torch.Size([3, 50, 50]) 3 --- torch.Size([1, 50, 50]) 3


#### CNN Layer 살펴보기

In [14]:
# 이미지 데이터로 conv2D
# 1. 흑백 이미지
con_Layer = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=0)
# - 커널은 3x3, 이미지 크기는 50x50
#   => 50 % 3 = 2 이므로 48만큼만 스캔, 48x48 출력

output = con_Layer(gray_hwc)
print(output.shape)     # torch.Size([1, 48, 48]) => 2만큼 짤린 것을 확인

torch.Size([1, 48, 48])


In [16]:
# out_channels를 8로 변경
con_Layer = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=0)
output = con_Layer(gray_hwc)
print(output.shape)     # torch.Size([8, 48, 48]) => 8개의 채널이 생성됨!!

torch.Size([8, 48, 48])


In [19]:
# conv2D 한 번 더
con_Layer1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=0)
con_Layer2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, stride=1, padding=0)
print(con_Layer1, con_Layer2, sep='\n')

Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))


In [22]:
# Padding의 차이 확인
con_Layer_p0 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=0)
con_Layer_p1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=1)
con_Layer_p2 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, stride=1, padding=2)
print(con_Layer_p0(gray_hwc).shape, con_Layer_p1(gray_hwc).shape, con_Layer_p2(gray_hwc).shape, sep='\n')
# - torch.Size([8, 48, 48])
# - torch.Size([8, 50, 50])
# - torch.Size([8, 52, 52])

torch.Size([8, 48, 48])
torch.Size([8, 50, 50])
torch.Size([8, 52, 52])


In [ ]:
# Kernel을 